# Action recognition using TAO ActionRecognitionNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train 3D RGB only for action recognition on the subset of [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .etlt file for deployment to DeepStream.

## Table of Contents

This notebook shows an example usecase of ActionRecognitionNet using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Inferences](#head-6)
7. [Deploy](#head-7)


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [1]:
import os

cwd = os.getcwd()

%env ROOT_DIR=$cwd
%env HOST_DATA_DIR=$cwd/data

# note: You could set the HOST_SPECS_DIR to folder of the experiments specs downloaded with the notebook
%env HOST_SPECS_DIR=$cwd/specs
%env HOST_RESULTS_DIR=$cwd/results

# Set your encryption key, and use the same key for all commands
%env KEY = nvidia_tao

env: ROOT_DIR=/mnt/d/Work/NVidia/action_recognition_net
env: HOST_DATA_DIR=/mnt/d/Work/NVidia/action_recognition_net/data
env: HOST_SPECS_DIR=/mnt/d/Work/NVidia/action_recognition_net/specs
env: HOST_RESULTS_DIR=/mnt/d/Work/NVidia/action_recognition_net/results
env: KEY=nvidia_tao


In [2]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR

In [2]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       },
       {
           "source": os.path.expanduser("~/.cache"),
           "destination": "/root/.cache"
       }
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [3]:
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "/mnt/d/Work/NVidia/action_recognition_net/data",
            "destination": "/data"
        },
        {
            "source": "/mnt/d/Work/NVidia/action_recognition_net/specs",
            "destination": "/specs"
        },
        {
            "source": "/mnt/d/Work/NVidia/action_recognition_net/results",
            "destination": "/results"
        },
        {
            "source": "/home/span/.cache",
            "destination": "/root/.cache"
        }
    ],
    "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
        }
    }
}

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.6.9 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.


In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [4]:
# View the versions of the TAO launcher
!tao info

Configuration of the TAO Toolkit Instance
dockers: ['nvidia/tao/tao-toolkit-tf', 'nvidia/tao/tao-toolkit-pyt', 'nvidia/tao/tao-toolkit-lm']
format_version: 2.0
toolkit_version: 3.22.05
published_date: 05/25/2022


## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

### 2.1 Prepare dataset

 We will be using the [HMDB51](https://serre-lab.clps.brown.edu/resource/hmdb-a-large-human-motion-database/) dataset for the tutorial. Download the HMDB51 dataset and unrar them firstly (We choose fall_floor/ride_bike for this tutorial): 

In [11]:
# install unrar
# NOTE: The following commands require `sudo`. You can run the command outside the notebook.
!apt update
!apt-get install unrar
!apt-get install unzip

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [ ]:
!pip install opendatasets

In [5]:
import opendatasets as od

In [ ]:
od.download("https://www.kaggle.com/datasets/odins0n/ucf-crime-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 

 spandoescode


Your Kaggle Key: 

 ································


100%|████████████████████████████████████████████████████████████████████| 11.0G/11.0G [14:35<00:00, 13.5MB/s]


Clone the dataset process script

In [5]:
!if [ -d tao_toolkit_recipes ]; then rm -rf tao_toolkit_recipes; fi
!git clone https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes

Cloning into 'tao_toolkit_recipes'...
error: chmod on /mnt/d/Work/NVidia/action_recognition_net/tao_toolkit_recipes/.git/config.lock failed: Operation not permitted
fatal: could not set 'core.filemode' to 'false'


Install the dependency for data generator:

In [ ]:
!pip3 install xmltodict opencv-python

Run the process script. 

In [ ]:
!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB_RGB.sh $HOST_DATA_DIR/raw_data $HOST_DATA_DIR/processed_data

We also provide scripts to preprocess optical flow dataset. The following cells for processing optical flow dataset is `Optional`.

`OPTIONAL:` Download the app based on NVOF SDK to generate optical flow. It is packaged with this notebook.

In [ ]:
#!echo <passwd> | sudo -S apt install -y libfreeimage-dev

`OPTIONAL` Run the process script for HMDB. 

`IMPORTANT NOTE`: to run the `preprocess_HMDB.sh` generating optical flow, a Turing or Ampere above GPU is needed. 

In [ ]:
#!cp ./AppOFCuda tao_toolkit_recipes/tao_action_recognition/data_generation/
#!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && bash ./preprocess_HMDB.sh $HOST_DATA_DIR/raw_data $HOST_DATA_DIR/processed_data

In [ ]:
# run split_HMDB to generate training split
!if [ -d $HOST_DATA_DIR/train ]; then rm -rf $HOST_DATA_DIR/train $HOST_DATA_DIR/test; fi
!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && python3 ./split_dataset_train.py $HOST_DATA_DIR/processed_data $HOST_DATA_DIR/splits/Action_Regnition_splits $HOST_DATA_DIR/train  $HOST_DATA_DIR/test
!cd tao_toolkit_recipes/tao_action_recognition/data_generation/ && python3 ./split_dataset_test.py $HOST_DATA_DIR/processed_data $HOST_DATA_DIR/splits/Action_Regnition_splits $HOST_DATA_DIR/train  $HOST_DATA_DIR/test

In [ ]:
# verify
!ls -l $HOST_DATA_DIR/train
!ls -l $HOST_DATA_DIR/train/Arson
!ls -l $HOST_DATA_DIR/test
!ls -l $HOST_DATA_DIR/test/NormalVideos

### 2.2 Download pretrained model from NGC

We will use NGC CLI to get the pre-trained models. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
import os
%env CLI=ngccli_cat_linux.zip
!mkdir -p $HOST_RESULTS_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $HOST_RESULTS_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $HOST_RESULTS_DIR/ngccli
!unzip -u "$HOST_RESULTS_DIR/ngccli/$CLI" -d $HOST_RESULTS_DIR/ngccli/
!rm $HOST_RESULTS_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("HOST_RESULTS_DIR", ""), os.getenv("PATH", ""))

In [ ]:
#!ngc registry model list nvidia/tao/actionrecognitionnet:*

In [18]:
!mkdir -p $HOST_RESULTS_DIR/pretrained

In [ ]:
# Pull pretrained model from NGC 
#!ngc registry model download-version "nvidia/tao/actionrecognitionnet:trainable_v1.0" --dest $HOST_RESULTS_DIR/pretrained
# Download the optical flow model from NGC
# !ngc registry model download-version "nvidia/tao/actionrecognitionnet:trainable_v2.0" --dest $HOST_RESULTS_DIR/pretrained

In [10]:
!wget --content-disposition https://api.ngc.nvidia.com/v2/models/nvidia/tao/actionrecognitionnet/versions/deployable_v1.0/zip -P $HOST_RESULTS_DIR/pretrained -O actionrecognitionnet_deployable_v1.0.zip
!unzip actionrecognitionnet_deployable_v1.0.zip -d $HOST_RESULTS_DIR/pretrained

--2022-07-05 16:27:16--  https://api.ngc.nvidia.com/v2/models/nvidia/tao/actionrecognitionnet/versions/deployable_v1.0/zip
Resolving api.ngc.nvidia.com (api.ngc.nvidia.com)... 52.9.61.239, 52.9.131.228, 205.251.193.211, ...
Connecting to api.ngc.nvidia.com (api.ngc.nvidia.com)|52.9.61.239|:443... connected.
HTTP request sent, awaiting response... 302 
Location: https://prod-model-registry-ngc-bucket.s3.us-west-2.amazonaws.com/org/nvidia/team/tao/models/actionrecognitionnet/versions/deployable_v1.0/files.zip?response-content-disposition=attachment%3B%20filename%3D%22files.zip%22&response-content-type=application%2Fzip&X-Amz-Security-Token=IQoJb3JpZ2luX2VjEHsaCXVzLXdlc3QtMSJIMEYCIQDEY8GzU8HA5%2F%2FtZi1mqoKHJv%2FRnMiBrIcuCIQdgDybuwIhAJgZ7rjQXPen64JcvlBvqG572IFKuFWmmR0oEKbiaE9pKtsECKT%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQBBoMNzg5MzYzMTM1MDI3IgzebGmIS8RVfUOKTGgqrwQv%2BiuMVdSLs3OU9pjNeA6dLu%2Fj78VKQ6AHKR2y9OqicGnlV%2Fq9TmPwTMlqK3bIPNCMlNyjNI4j%2FGxs9dN%2FyFa%2FzAG8RIwMCBqwC%2BqMoEszOQLoRUBLAz2wIG

In [21]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_RESULTS_DIR/pretrained/

Check that model is downloaded into dir.
total 347944
-rw-r--r-- 1 root root  45467175 May 17 00:31 resnet18_2d_of_hmdb5_32_a100.etlt
-rw-r--r-- 1 root root  45467175 May 17 00:31 resnet18_2d_of_hmdb5_32_xavier.etlt
-rw-r--r-- 1 root root 132673829 May 17 00:31 resnet18_3d_of_hmdb5_32_a100.etlt
-rw-r--r-- 1 root root 132673829 May 17 00:31 resnet18_3d_of_hmdb5_32_xavier.etlt


## 3. Provide training specification <a class="anchor" id="head-2"></a>

We provide specification files to configure the training parameters including:

* model_config: configure the model setting
    * model_type: type of model, rgb/of/joint
    * backbone: resnet18/34/50/101/152 
    * rgb_seq_length: length of RGB input sequence
    * input_type: 2d/3d
    * sample_strategy: consecutive
    * dropout_ratio: probability to drop the hidden units
* train_config: configure the training hyperparameters
    * optim_config
    * epochs
    * checkpoint_interval
* dataset_config: configure the dataset and augmentation methods
    * train_dataset_dir
    * val_dataset_dir
    * label_map: map the class label to id
    * output_shape
    * batch_size
    * workers: number of workers to do data loading
    * clips_per_video: number of clips to be sampled from single video
    * augmentation_config

Please refer to the TAO documentation about ActionRecognitionNet to get all the parameters that are configurable.

In [5]:
!cat $HOST_SPECS_DIR/train_rgb_3d_finetune.yaml

output_dir: /results/rgb_3d_ptm
encryption_key: nvidia_tao
model_config:
  model_type: rgb
  backbone: resnet18
  rgb_seq_length: 3
  input_type: 3d
  sample_strategy: consecutive
  dropout_ratio: 0.0
train_config:
  optim:
    lr: 0.001
    momentum: 0.9
    weight_decay: 0.0001
    lr_scheduler: MultiStep
    lr_steps: [5, 15, 20]
    lr_decay: 0.1
  epochs: 20
  checkpoint_interval: 1
dataset_config:
  train_dataset_dir: /data/train
  val_dataset_dir: /data/test
  label_map:
    abuse: 0
    arrest: 1
    arson: 2
    assault: 3
    burglary: 4
    explosion: 5
    fighting: 6    
    normal: 7
    roadaccident: 8
    robbery: 9
    shooting: 10
    shoplifting: 11
    stealing: 12
    vandalism: 13
  output_shape:
  - 64
  - 64
  batch_size: 32
  workers: 8
  clips_per_video: 5
  augmentation_config:
    train_crop_type: no_crop
    horizontal_flip_prob: 0.5
    rgb_input_mean: [0.5]
    rgb_input_std: [0.5]
    val_center_crop: False


## 4. Run TAO training <a class="anchor" id="head-3"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [6]:
# NOTE: The following paths are set from the perspective of the TAO Docker.

# The data is saved here
%env DATA_DIR = /data
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results

env: DATA_DIR=/data
env: SPECS_DIR=/specs
env: RESULTS_DIR=/results


### 4.1 Train 3D model:

We provide pretrained RGB-only model trained on HMDB5 dataset. With the pretrained model, we can even get better accuracy with less epochs.

`KNOWN ISSUE`: 
- 1) The training log will be corrupted by pytorch warning in the notebook:

     `[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)` 
     
     To see the full log in std out, please run the command in terminal. 
- 2) "=" in the checkpoint file name should removed before using the checkpoint in command.

In [7]:
!docker login nvcr.io

Authenticating with existing credentials...
Login Succeeded


In [8]:
print("Train RGB only model with PTM")
!tao action_recognition train \
                  -e $SPECS_DIR/custom_train_rgb_3d_finetune.yaml \
                  -r $RESULTS_DIR/rgb_3d_ptm \
                  -k $KEY \
                  model_config.rgb_pretrained_model_path=$RESULTS_DIR/pretrained/resnet18_3d_rgb_hmdb5_32.etlt  \
                  model_config.rgb_pretrained_num_classes=5

Train RGB only model with PTM
2022-07-06 17:05:05,412 [INFO] root: Registry: ['nvcr.io']
2022-07-06 17:05:05,610 [INFO] tlt.components.instance_handler.local_instance: Running command in container: nvcr.io/nvidia/tao/tao-toolkit-pyt:v3.22.05-py3
2022-07-06 17:05:05,639 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/span/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Docker instantiation failed with error: 500 Server Error: Internal Server Error ("failed to create shim task: OCI runtime create failed: runc create failed: unable to start container process: error during container init: error running hook #0: error running hook: exit status 1, stdout: , stderr: nvidia-container-cli: mount error: file creation failed: /var/lib/docker/overlay

In [ ]:
print('Encrypted checkpoints:')
print('---------------------')
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d_ptm

In [ ]:
print('Rename a model: Note that the training is not deterministic, so you may change the model name accordingly.')
print('---------------------')
# NOTE: The following command may require `sudo`. You can run the command outside the notebook.
!find $HOST_RESULTS_DIR/rgb_3d_ptm/ -name *epoch=19* | xargs realpath | xargs -I {} mv {} $HOST_RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt 
!ls -ltrh $HOST_RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt

### `OPTIONAL` 4.2 Train optical flow only model

`Important Note` The following cells are using optical flow dataset. 

We will train a 3D OF-only model

In [ ]:
# print("Train 3D OF-only model")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_of_3d_finetune.yaml \
#                   -r $RESULTS_DIR/of_3d_ptm \
#                   -k $KEY \
#                   dataset_config.train_dataset_dir=$DATA_DIR/train \
#                   dataset_config.val_dataset_dir=$DATA_DIR/test \
#                   model_config.of_pretrained_model_path=$RESULTS_DIR/pretrained/actionrecognitionnet_vtrainable_v2.0/resnet18_3d_of_hmdb5_32_a100.tlt  \
#                   model_config.of_pretrained_num_classes=5

In [ ]:
# print("To resume training from a checkpoint, set the resume_training_checkpoint_path option to be the .tlt you want to resume from")
# print("remember to remove the `=` in the checkpoint's file name")
# !tao action_recognition train \
#                   -e $SPECS_DIR/train_of_3d.yaml \
#                   -r $RESULTS_DIR/of_3d_ptm \
#                   -k $KEY \
#                   resume_training_checkpoint_path=

In [ ]:
# print('Encrypted checkpoints:')
# print('---------------------')
# !ls -ltrh $HOST_RESULTS_DIR/of_3d_ptm

In [ ]:
# print('Rename a model: ')
# print('---------------------')
# # NOTE: The following command may require `sudo`. You can run the command outside the notebook.
# !find $HOST_RESULTS_DIR/of_3d_ptm/ -name *epoch=19* | xargs realpath | xargs -I {} mv {} $HOST_RESULTS_DIR/of_3d_ptm/of_only_model.tlt 
# !ls -ltrh $HOST_RESULTS_DIR/of_3d_ptm/of_only_model.tlt

## 5. Evaluate trained models <a class="anchor" id="head-4"></a>

We provide two different sample strategy to evaluate the pretrained model on video clips.

* `center` mode: pick up the middle frames of a sequence to do inference. For example, if the model requires 32 frames as input and a video clip has 128 frames, then we will choose the frames from index 48 to index 79 to do the inference. 
* `conv` mode: convolutionly sample 10 sequences out of a single video and do inference. The final results are averaged.

Evaluate RGB model trained with PTM

In [ ]:
!tao action_recognition evaluate \
                    -e $SPECS_DIR/evaluate_rgb.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt  \
                    batch_size=1 \
                    test_dataset_dir=$DATA_DIR/test \
                    video_eval_mode=center

`Optional:` Evaluate OF model

In [ ]:
# !tao action_recognition evaluate \
#                     -e $SPECS_DIR/evaluate_of.yaml \
#                     -k $KEY \
#                     model=$RESULTS_DIR/of_3d_ptm/of_only_model.tlt  \
#                     batch_size=1 \
#                     test_dataset_dir=$DATA_DIR/test \
#                     video_eval_mode=center

## 6. Inferences <a class="anchor" id="head-5"></a>
In this section, we run the action recognition inference tool to generate inferences with the trained RGB models and print the results. 

There are also two modes for inference just like evaluation: `center` mode and `conv` mode. And the final output will show each input sequence label in the videos like:
`[video_sample_path] [labels list for sequences in the video sample]`

In [ ]:
!tao action_recognition inference \
                    -e $SPECS_DIR/infer_rgb.yaml \
                    -k $KEY \
                    model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt \
                    inference_dataset_dir=$DATA_DIR/test/ride_bike \
                    video_inf_mode=center

`Optional:` Inference with OF-only model

In [ ]:
# !tao action_recognition inference \
#                     -e $SPECS_DIR/infer_of.yaml \
#                     -k $KEY \
#                     model=$RESULTS_DIR/of_3d_ptm/of_only_model.tlt \
#                     inference_dataset_dir=$DATA_DIR/test/ride_bike \
#                     video_inf_mode=center

## 7. Deploy! <a class="anchor" id="head-6"></a>

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/export

In [ ]:
# Export the RGB model to encrypted ONNX model
!tao action_recognition export \
                   -e $SPECS_DIR/export_rgb.yaml \
                   -k $KEY \
                   model=$RESULTS_DIR/rgb_3d_ptm/rgb_only_model.tlt\
                   output_file=$RESULTS_DIR/export/rgb_resnet18_3.etlt

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/export

This notebook has come to an end. You may continue by deploying this RGB model to [DeepStream](https://docs.nvidia.com/metropolis/deepstream/dev-guide/text/DS_3D_Action.html)

`Optional` Export OF-Only model

In [ ]:
# # Export the OF model to encrypted ONNX model
# !tao action_recognition export \
#                    -e $SPECS_DIR/export_of.yaml \
#                    -k $KEY \
#                    model=$RESULTS_DIR/of_3d_ptm/of_only_model.tlt\
#                    output_file=$RESULTS_DIR/export/of_resnet18_3.etlt

OF model is not supported in DeepStream. But you can play with stand-alone TensorRT inference in [tao_toolkit_recipes](https://github.com/NVIDIA-AI-IOT/tao_toolkit_recipes/tree/main/tao_action_recognition/tensorrt_inference) 